In [4]:
import numpy as np
import scipy.integrate as integrate
from scipy.signal import fftconvolve
import os,sys
import yaml
import matplotlib.pyplot as plt
import math

In [5]:
with open('configure.yml','r') as conf_para:
    conf_para = yaml.load(conf_para,Loader=yaml.FullLoader)
#print(conf_para)

In [6]:
def wavefront_initialize(pixelsize_x = 55e-06,pixelsize_y = 55e-06,fs_size = 2000,ss_size = 2000,focus_x = 1.2e-3,focus_y = 1.0e-3,defocus = 400e-6, det_dist = 14e-03, ap_x = 40e-06, ap_y= 40e-6,wl = 7.29e-11,amplitude_value=0.0):

    #wf_dec = np.zeros((ss_size,fs_size),dtype='complex')
    #wf_dec += amplitude_value


    # the range of detector plane(x-axis,y-axis)
    xx_span = fs_size * pixelsize_x
    yy_span = ss_size * pixelsize_y

    # the range of object plane(x-axis,y-axis)
    x_span = 2* ap_x / focus_x * defocus
    y_span = 2* ap_y / focus_y * defocus
    # the sample rate in the object plane
    n_x = int(x_span * xx_span / wl / det_dist)
    n_y = int(y_span * yy_span / wl / det_dist)

    # Initializing coordinate arrays
    # coordinate in object plane
    x_arr = np.linspace(-x_span / 2, x_span / 2, n_x)
    y_arr = np.linspace(-y_span / 2, y_span / 2, n_y)
    

    #wf_ini = np.zeros((n_x,n_y),dtype='complex')

    # coordinate in detector plan
    xx_arr = np.linspace(-xx_span / 2, xx_span / 2, fs_size, endpoint=False)
    yy_arr = np.linspace(-yy_span / 2, yy_span / 2, ss_size, endpoint=False)

    return x_arr,y_arr,xx_arr,yy_arr,n_x,n_y

In [7]:
x_arr,y_arr,xx_arr,yy_arr,n_x,n_y = wavefront_initialize(pixelsize_x = 55e-06,pixelsize_y = 55e-06,fs_size = 2000,ss_size = 2000,focus_x = 1.2e-3,focus_y = 1.0e-3,defocus = 400e-6, det_dist = 14e-03, ap_x = 40e-06, ap_y= 40e-6,wl = 7.29e-11,amplitude_value=0.0)

In [8]:
alpha_x = conf_para['Lens']['alpha_x']
alpha_y = conf_para['Lens']['alpha_y']
focus_x = conf_para['Lens']['focus_x']
focus_y = conf_para['Lens']['focus_y']

det_dist = conf_para['Exp_geom']['det_dist']
defocus = conf_para['Exp_geom']['defocus']
ap_x = conf_para['Lens']['ap_x']
ap_y = conf_para['Lens']['ap_y']

wl = 7.29e-11
k = 2*np.pi/wl
n_arr = np.arange(-n_x // 2, n_x // 2)
x0_arr = n_arr/n_x*ap_x*2
u0_x = np.exp(1.j*k/2*x0_arr**2 + 1e9j*alpha_x*(x0_arr/focus_x)**3)
u0_x[np.abs(x0_arr)>ap_x/2] = 0

%matplotlib widget
plt.plot(x0_arr,u0_x)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
delta_x = ap_x / n_x
h_x = delta_x * (focus_x + defocus) / 1.j / wl**0.5 * np.exp(-1.j * k * np.sqrt(x0_arr**2 + (focus_x + defocus)**2)) / (x0_arr**2 + (focus_x + defocus)**2)**0.75

u0_f_x = np.fft.fft(u0_x)

h_f_x = np.fft.fft(h_x)
h_f_x = np.fft.fftshift(h_f_x)
u0_f_x = np.fft.fftshift(u0_f_x)

#fx_arr = np.fft.fftfreq(n_x)*n_x
#g_x = np.fft.ifft(np.exp(-1.j * np.pi * fx_arr**2 / n_x)*h_f_x*u0_f_x)
#h_x = np.fft.ifft(np.exp(1j * np.pi / n_x * fx_arr**2 /n_x))
#u1_x = np.fft.fftshift(np.exp(-1.j*np.pi*fx_arr**2 /n_x)*np.fft.fft(g_x*h_x))


In [10]:

fa = 1
wm_x = np.exp(-1j * np.pi * x_arr**2 * fa / n_x)
g_x = np.exp(-1j * np.pi * x_arr**2 * fa / n_x)*h_f_x*u0_f_x
h_x = np.exp(1j * np.pi * x_arr**2 * fa / n_x)
u1_x = wm_x * fftconvolve(g_x,h_x,mode='same')

print(u1_x.shape)
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
#axes[0].plot(np.abs(u1_x[500:2200]))
axes[0].plot(np.abs(u1_x))
axes[0].set_title('Lens_x Amplitude', fontsize=20)

#axes[1].plot(np.unwrap(np.imag(u1_x[500:2200])))
axes[1].plot(np.unwrap(np.imag(u1_x)))
axes[1].set_title('Lens_x Phase', fontsize=20)
#plt.savefig('Lens_x_1D.png')

(2874126,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Lens_x Phase')

In [12]:
y_lens = 2*n_arr/n_y*ap_y
u0_y = np.exp(1.j*k/2*y_lens**2/(defocus) + 1e9j*alpha_y*(y_lens/focus_y)**3)

delta_y = ap_y / n_y

h_y = delta_y * (focus_y + defocus) / 1.j / wl**0.5 * np.exp(-1.j * k * np.sqrt(y_lens**2 + (focus_y + defocus)**2)) / (y_lens**2 + (focus_y + defocus)**2)**0.75



u0_f_y = np.fft.fft(u0_y)
h_f_y = np.fft.fft(h_y)
h_f_y = np.fft.fftshift(h_f_y)
u0_f_y = np.fft.fftshift(u0_f_y)



aa_y = 1
wm_y = np.exp(1.j * np.pi / n_y * y_arr**2 *aa_y /2)
hm_y = np.exp(-1j * np.pi * y_arr**2 * aa_y /2 / n_y)

u1_y = wm_y * fftconvolve(wm_y*h_y*u0_y,hm_y,mode='same')

%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(np.real(u1_y))
axes[0].set_title('Lens_y Amplitude', fontsize=20)
axes[1].plot(np.unwrap(np.imag(u1_y)))
axes[1].set_title('Lens_y Phase', fontsize=20)
plt.savefig('Lens_y_1D.png')



ValueError: operands could not be broadcast together with shapes (3448951,) (2874126,) 

In [15]:

h_y_new = np.exp(1.j* k /2 /(defocus) * y_lens**2)

h_f_ynew = np.fft.fft(h_y_new)
h_f_ynew = np.fft.fftshift(h_f_ynew)
print(h_f_ynew.shape)
u_0_try = np.ones((h_f_ynew.shape[0]))
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(y_lens)
axes[0].set_title('Amplitude', fontsize=20)
axes[1].plot(np.unwrap(np.imag(u_0_try)))
axes[1].set_title('Lens_y Phase', fontsize=20)




(2874126,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Lens_y Phase')

In [16]:
h_y_new = np.exp(1.j*k/2 /(defocus*1000) * y_lens**2)
u_0_try = np.ones((10000))
u_0_try = np.pad(u_0_try,(10000,10000))
print(u_0_try)
plt.plot(u_0_try)
print(u_0_try.shape)



[0. 0. 0. ... 0. 0. 0.]
(30000,)


In [18]:
h_y_new = np.pad(h_y_new,(10,10))
print(h_y_new.shape)
u_try = np.convolve(h_y_new,u_0_try,mode = 'same')
#u_try = fftconvolve(h_y_new,u_0_try,mode='same')
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(np.abs(u_try))
axes[0].set_title('Lens_y Amplitude', fontsize=20)
axes[1].plot(np.unwrap(np.imag(u_try)))
axes[1].set_title('Lens_y Phase', fontsize=20)

(2874166,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Lens_y Phase')

In [181]:
wf_lens = np.zeros((len(u1_y),len(u1_x)),dtype='complex')
u1_x = u1_x.reshape(1,len(u1_x))
u1_y
        wf_lens[i][j] = u1_x[j]*u1_y[i]


In [182]:
print(wf_lens.shape)
wf_lens_new = wf_lens
if len(u1_y)<len(u1_x):
    diff = np.abs(len(u1_x)-len(u1_y))//2
else:

    diff = np.abs(len(u1_x)-len(u1_y))//2

(3448, 2874)


In [183]:
wf_lens_new = wf_lens[diff:(len(wf_lens)-diff),:]
print(wf_lens_new[0])

[   358071.46491085-14794363.20927467j
   5224096.08331366-14612463.10137892j
   2750836.52128385-16516559.89901436j ...
 -13718904.09453787 -7700508.34094232j
  -8595890.02987994 -5516583.36549783j
  -4709610.2780952  -5702556.14279214j]


In [205]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
real = axes[0].imshow(np.real(wf_lens_new[50:2500,50:2500]))
axes[0].set_title('Amplitude on defocus plane', fontsize=20)

imag = axes[1].imshow(np.unwrap(np.imag(wf_lens_new[50:2500,50:2500])))
axes[1].set_title('Phase on defocus plane', fontsize=20)
plt.savefig('wf_defocus_wrap.png')


In [184]:
print(len(u1_x),len(np.real(u1_x)))

2874 2874


In [23]:
print(wf_lens[1])

[ -2999860.69861899-15420413.17393727j
   2086690.62906887-16340612.97461795j
   -911465.18808825-17751215.84034516j ...
 -15978455.82852454 -4857849.50275963j
 -10169267.05122465 -3760905.21754768j
  -6182342.06075888 -4839267.21274496j]


In [114]:
print(x_arr.shape,y_arr.shape,x_lens.shape,y_lens.shape)
diff = np.abs(x_lens.shape[0]-y_lens.shape[0])//2
print(diff//2)

if x_arr.shape[0]<y_arr.shape[0]:
    #x_arr = np.pad(x_arr,(diff//2,diff//2),'constant',constant_values = 0)
    y_arr_new = y_arr[diff:(len(y_arr)-diff),]
elif x_arr.shape[0]>y_arr.shape[0]:
    #y_arr = np.pad(y_arr,(diff//2,diff//2),'constant',constant_values = 0)
    x_arr_new = np.asarray(x_arr[diff:(len(x_arr)-diff),])

#print((x_arr_new.shape),(y_arr_new.shape),(x_lens.shape),(y_lens.shape))
y = y_arr[diff:(len(y_arr)-diff)]
print(y)
print(y_arr_new)
print(x_arr_new)
y_arr[diff:(len(y_arr)-diff)]


(2874,) (3448,) (2874,) (3448,)
143
[-1.33356542e-05 -1.33263708e-05 -1.33170873e-05 ...  1.33170873e-05
  1.33263708e-05  1.33356542e-05]
[-1.33356542e-05 -1.33263708e-05 -1.33170873e-05 ...  1.33170873e-05
  1.33263708e-05  1.33356542e-05]
[0. 0. 0. ... 0. 0. 0.]


array([-1.33356542e-05, -1.33263708e-05, -1.33170873e-05, ...,
        1.33170873e-05,  1.33263708e-05,  1.33356542e-05])

In [165]:
def aperture_type(x_arr,y_arr,wf_ini,ap_x = 40e-06,ap_y = 40e-06,type =0,sigma = 2.35,focus_x = 1.2e-3,focus_y = 1.2e-3, defocus = 400e-06):
    ap_xx = x_arr[:,np.newaxis]
    ap_yy = y_arr[np.newaxis,:]
    diameter_x = ap_x/focus_x*defocus/5
    diameter_y = ap_y/focus_y*defocus/5
    filter = np.zeros_like(wf_ini)
    if type == 0: # Rectangular aperture_type
        radius_x = (diameter_x/2)
        radius_y = (diameter_y/2)
        print(radius_x,radius_y)
        filter_illuminated_indices = np.where((np.abs(ap_xx)>radius_x) & (np.abs(ap_yy)>radius_y))
        print(filter_illuminated_indices)
        filter[filter_illuminated_indices] = 0.0
    elif type == 1: # Circular aperture_type
        radius = np.sqrt((diameter_x/2)**2 + (diameter_y/2)**2)
        print(radius)
        filter_illuminated_indices = np.where((ap_xx**2 + ap_yy**2)<radius**2)
        filter[filter_illuminated_indices] = 1.0
    elif type == 2: # Gaussian_type
        sigma_x = diameter_x/2.35
        sigma_y = diameter_y/2.35
        filter = np.sqrt(np.exp(-pow(ap_xx/sigma_x,2)/2-pow((ap_yy/sigma_y),2)/2))
    return wf_ini + filter

In [166]:
wf_obj = aperture_type(x_arr,y_arr_new,wf_lens_new,type=0)
print(max(x_arr))

1.3333333333333336e-06 1.3333333333333336e-06
(array([   0,    0,    0, ..., 2873, 2873, 2873]), array([   0,    1,    2, ..., 2871, 2872, 2873]))
1.3333333333333337e-05


In [43]:
print(y_arr_new.shape,x_arr.shape,wf_lens_f.shape)

%matplotlib widget

#plt.imshow(np.unwrap(np.imag(wf_lens_new)))
#plt.imshow(np.unwrap(np.imag(wf_obj)))
plt.imshow(np.real(wf_obj))



NameError: name 'y_arr_new' is not defined

In [218]:
pixelsize_x = conf_para['Detector']['pixelsize_x']
pixelsize_y = conf_para['Detector']['pixelsize_y']
delta_y = ap_y / n_y

cor_obj = np.asarray(np.meshgrid(y_arr_new,x_arr))
print(cor_obj.shape)
#h_r = delta_y * delta_x * det_dist / 1.j / wl**0.5 * np.exp(-1.j * k * np.sqrt(y_arr_new**2 + x_arr**2 +(det_dist)**2)) / (y_arr_new**2 + x_arr**2 + (det_dist)**2)**0.75
h_r = delta_y * delta_x * det_dist / 1.j / wl**0.5 * np.exp(-1.j * k * np.sqrt(cor_obj[0]**2 +cor_obj[1]**2+(det_dist)**2)) / (cor_obj[0]**2 + cor_obj[1]**2+(det_dist)**2)**0.75

print(h_r.shape)
wf_lens_f = np.fft.fft(wf_lens_new)
h_f_r = np.fft.fft(h_r)
h_f_r = np.fft.fftshift(h_f_r)
wf_lens_f = np.fft.fftshift(wf_lens_f)

aa_rx = delta_x/pixelsize_x
aa_ry = delta_y/pixelsize_y

if xx_arr.shape[0]<x_arr.shape[0]:
    compliment_X = np.abs(x_arr.shape[0] - xx_arr.shape[0])//2
else:
    print("No need to pad 0 .")
xx_arr_new = np.pad(xx_arr,(compliment_X,compliment_X),'constant',constant_values = 0)

if yy_arr.shape[0]<y_arr_new.shape[0]:
    compliment_Y = np.abs(y_arr_new.shape[0] - yy_arr.shape[0])//2
else:
    print("No need to pad 0 .")

yy_arr_new = np.pad(yy_arr,(compliment_Y,compliment_Y),'constant',constant_values = 0)

cor_dec = np.asarray(np.meshgrid(yy_arr_new,xx_arr_new))
wm_r = np.exp((1.j * np.pi / (len(yy_arr)+len(xx_arr))* (cor_dec[0]**2+cor_dec[1]**2 ) *(aa_ry+aa_rx) /2))
print(wm_r.shape)
hm_r = np.exp(-1j * np.pi * ( (len(yy_arr)+len(xx_arr)) *  (cor_dec[0]**2+cor_dec[1]**2 )*(aa_ry+ aa_rx )/2 ))

wf_dec = wm_r * fftconvolve(wm_r*h_f_r*wf_lens_f,hm_r,mode='same')

%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].imshow(np.abs(wf_dec)**2)
axes[0].set_title('Detector Amplitude', fontsize=20)
axes[1].imshow(np.unwrap(np.imag(wf_dec)))
axes[1].set_title('Detector Phase', fontsize=20)
plt.savefig('Detector.png')

(2, 2874, 2874)
(2874, 2874)
(2874, 2874)


In [128]:
print(wf_lens_f.shape)

(2874, 2874)
